In [1]:
import numpy as np
from collections.abc import Callable

In [2]:
def testfunction(x: np.ndarray) -> np.ndarray:
    """Example function for testing Newton's method for systems of equations. Taken from Textbook, Section 10.1, Example 1.

    Args:
        x (np.ndarray): Input array of shape (2,).

    Returns:
        np.ndarray: Output array of shape (2,).
    """
    return np.array([3*x[0]-np.cos(x[1]*x[2]) -0.5,
                     x[0]**2 - 81*(x[1]+0.1)**2 + np.sin(x[2]) + 1.06, 
                     np.exp(-x[0]*x[1]) + 20*x[2] + (10*np.pi-3)/3])

In [3]:
def jacobian_testfunction(x: np.ndarray) -> np.ndarray:
    """Jacobian of the example function for testing Newton's method for systems of equations. Taken from Textbook, Section 10.1, Example 1.

    Args:
        x (np.ndarray): Input array of shape (2,).

    Returns:
        np.ndarray: Output array of shape (2,2
    """
    return np.array([[3, x[2]*np.sin(x[1]*x[2]), x[1]*np.sin(x[1]*x[2])],
                     [2*x[0],-162*(x[1]+0.1), np.cos(x[2])],
                     [- x[1]*np.exp(-x[0]*x[1]), -x[0]*np.exp(-x[0]*x[1]), 20]])                     

In [4]:
def newton(
    func: Callable[[np.ndarray], np.ndarray], jacobian: Callable[[np.ndarray], np.ndarray], x: np.ndarray, eps: float = 1.0e-10, N_max = 100
) -> np.ndarray:
    """
    Newton-Raphson method for systems

    Arguments:
        func: The function for which the root is to be found
        jacobian: The Jacobian matrix of the function
        x: initial guess
        eps: convergence tolerance (default 1.0e-10)
        N_max: maximum number of iterations (default 100)

    Returns:
        x: the root approximation
    """

    f_val = func(x)
    counter = 0

    print(f"{counter:4d} {x[0]:.6e} {np.linalg.norm(x, ord=np.inf):.6e}")

    while np.linalg.norm(f_val,ord=np.inf) > eps:

        # note: in general, this is not the most efficient way to solve the linear system
        # we discuss this in the next lecture

        x -= np.linalg.inv(jacobian(x))@ f_val
        f_val = func(x)

        counter += 1
        print(f"{counter:4d} {x[0]:.6e} {np.linalg.norm(f_val, ord=np.inf):.6e}")
        
        if counter >= N_max:
            raise RuntimeError(f"Newton iteration failed to converge after {N_max} iterations")

    return x

In [5]:
try:
    x = newton(testfunction, jacobian_testfunction, np.array([0.1,0.1,-0.1]), eps=1e-10)
except RuntimeError as e:
    print(f"Runtime Error: {e}")

   0 1.000000e-01 1.000000e-01
   1 4.998697e-01 3.443879e-01
   2 5.000142e-01 2.588914e-02
   3 5.000001e-01 2.012227e-04
   4 5.000000e-01 1.254308e-08
   5 5.000000e-01 1.776357e-15


In [6]:
print(f"Root of testfunction: {x}")

Root of testfunction: [ 5.00000000e-01  6.82466595e-18 -5.23598776e-01]


# Broyden's Method

In [7]:
def broyden(
    func: Callable[[np.ndarray], np.ndarray], jacobian: Callable[[np.ndarray], np.ndarray], x: np.ndarray, eps: float = 1.0e-10, N_max = 100
) -> np.ndarray:
    """
    Broyden's method for systems

    Arguments:
        func: The function for which the root is to be found
        jacobian: The Jacobian matrix of the function
        x: initial guess
        eps: convergence tolerance (default 1.0e-10)
        N_max: maximum number of iterations (default 100)

    Returns:
        x: the root approximation
    """
    A0 = jacobian(x)
    v = func(x)
    counter = 0
    print(f"{counter:4d} {x[0]:.6e} {np.linalg.norm(v, ord=np.inf):.6e}")
    A = np.linalg.inv(A0)
    s = - A @ v
    x += s
    counter += 1
    print(f"{counter:4d} {x[0]:.6e} {np.linalg.norm(func(x), ord=np.inf):.6e}")

    while np.linalg.norm(v,ord=np.inf) > eps:

        w = np.copy(v)
        v = func(x)
        y = v - w
        z = - A @ y
        p = -np.dot(s,z)

        A = A + np.outer((s +z), np.transpose(A) @ s) / p
        s = - A @ v
      
        x += s

        counter += 1
        print(f"{counter:4d} {x[0]:.6e} {np.linalg.norm(v, ord=np.inf):.6e}")
        
        if counter >= N_max:
            raise RuntimeError(f"Broyden iteration failed to converge after {N_max} iterations")


    return x

In [8]:
try:
    x = broyden(testfunction, jacobian_testfunction, np.array([0.1,0.1,-0.1]), eps=1e-10)    
except RuntimeError as e:
    print(f"Runtime Error: {e}")

   0 1.000000e-01 8.462025e+00
   1 4.998697e-01 3.443879e-01
   2 4.999864e-01 3.443879e-01
   3 5.000066e-01 1.473835e-01
   4 5.000003e-01 1.408127e-02
   5 5.000000e-01 6.392117e-04
   6 5.000000e-01 3.129052e-06
   7 5.000000e-01 1.633715e-11


In [9]:
print(f"Root of testfunction: {x}")

Root of testfunction: [ 5.00000000e-01  1.66505285e-13 -5.23598776e-01]


# Steepest Descent Method

In [10]:
def SteepestDescent (func: Callable[[np.ndarray], np.ndarray], jacobian: Callable[[np.ndarray], np.ndarray], x: np.ndarray, eps: float = 1.0e-08, N_max = 500) -> np.ndarray:
    """
    Steepest Descent method for systems of equations. 
    
    Note: here we implement the method to minimize the sum of 
    squares of a vector-valueed function (i.e., we find the 
    root of that function). You may want to consider a more 
    general implementation.

    Arguments:
        func: The function for which the root is to be found
        jacobian: The Jacobian matrix of the function
        x: initial guess
        eps: convergence tolerance (default 1.0e-08)
        N_max: maximum number of iterations (default 100)

    Returns:
        x: the root approximation
    """
    for k in range(N_max):
        g1 = sum(func(x)**2)
        grad_g1 = 2 * jacobian(x).T @ func(x)
        g1_norm = np.linalg.norm(grad_g1)

        if (g1_norm < eps):
            return x
        
        grad_g1 = grad_g1 / g1_norm
        alpha_3 = 1.0
        g3 = sum(func(x - alpha_3 * grad_g1)**2)

        while (g3 >= g1):
            alpha_3 = alpha_3 / 2.0
            g3 = sum(func(x - alpha_3 * grad_g1)**2)
            if (alpha_3 < 1.0e-10):
                raise RuntimeError("No more improvement possible in line search. We may have converged.")
        
        alpha_2 = alpha_3 / 2.0
        g2 = sum(func(x - alpha_2 * grad_g1)**2)

        # the next four lines interpolate g to a quadratic polynomial and compute its minimizer
        # we will learn later how to do this

        h_1 = (g2 - g1) / alpha_2
        h_2 = (g3 - g2) / (alpha_3 - alpha_2)
        h_3 = (h_2 - h_1) / alpha_3
        alpha_0 = 0.5 * (alpha_2 - h_1 / h_3)

        g0 = sum(func(x - alpha_0 * grad_g1)**2)
        if (g0 < g1 and g0 < g2 and g0 < g3):
            alpha = alpha_0
        elif (g2 < g1 and g2 < g3):
            alpha = alpha_2
        else:
            alpha = alpha_3
        x = x - alpha * grad_g1
        print(f"{k:4d} {x[0]:.6e}  {x[1]:.6e}  {x[2]:.6e} {g1_norm:.6e}")

    raise RuntimeError(f"Steepest Descent iteration failed to converge after {N_max} iterations")        




In [11]:
try:
    x = SteepestDescent(testfunction, jacobian_testfunction, np.array([0.0,0.0,0.0]))    
except RuntimeError as e:
    print(f"Runtime Error: {e}")

   0 1.121817e-02  1.009636e-02  -5.227408e-01 4.195538e+02
   1 1.378597e-01  -2.054528e-01  -5.220594e-01 1.740587e+01
   2 2.669594e-01  5.511020e-03  -5.584945e-01 1.284970e+01
   3 2.727338e-01  -8.117510e-03  -5.220061e-01 3.073916e+01
   4 3.086893e-01  -2.040263e-02  -5.331116e-01 4.560920e+00
   5 3.143082e-01  -1.470464e-02  -5.209234e-01 8.525531e+00
   6 3.242667e-01  -8.525489e-03  -5.284308e-01 4.542643e+00
   7 3.308088e-01  -9.678484e-03  -5.206623e-01 4.978682e+00
   8 3.398086e-01  -8.591975e-03  -5.280802e-01 3.942736e+00
   9 3.457463e-01  -9.033983e-03  -5.209410e-01 4.527691e+00
  10 3.539434e-01  -7.910567e-03  -5.276886e-01 3.594063e+00
  11 3.593566e-01  -8.346770e-03  -5.211855e-01 4.128200e+00
  12 3.668272e-01  -7.289804e-03  -5.273326e-01 3.276515e+00
  13 3.717620e-01  -7.702381e-03  -5.214067e-01 3.764084e+00
  14 3.785694e-01  -6.708684e-03  -5.270066e-01 2.987348e+00
  15 3.830688e-01  -7.098725e-03  -5.216070e-01 3.431890e+00
  16 3.892721e-01  -6.1671

In [12]:
print(f"Root of testfunction: {x}")

Root of testfunction: [ 5.00000000e-01  1.66505285e-13 -5.23598776e-01]
